Licensed under the Apache License, Version 2.0 (the "License");
<!--
    Licensed to the Apache Software Foundation (ASF) under one
    or more contributor license agreements.  See the NOTICE file
    distributed with this work for additional information
    regarding copyright ownership.  The ASF licenses this file
    to you under the Apache License, Version 2.0 (the
    "License"); you may not use this file except in compliance
    with the License.  You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing,
    software distributed under the License is distributed on an
    "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
    KIND, either express or implied.  See the License for the
    specific language governing permissions and limitations
    under the License.
-->


# Run Beam SQL in notebooks

[Beam SQL](https://beam.apache.org/documentation/dsls/sql/overview/) allows a Beam user to query PCollections with SQL statements. Currently, `InteractiveRunner` does not support `SqlTransform` due to [BEAM-10708](https://issues.apache.org/jira/browse/BEAM-10708). However, a user could use the `beam_sql` magic to run Beam SQL in the notebook and introspect the result.

`beam_sql` is an IPython [custom magic](https://ipython.readthedocs.io/en/stable/config/custommagics.html). If you're not familiar with magics, here are some [built-in examples](https://ipython.readthedocs.io/en/stable/interactive/magics.html). It's a convenient way to validate your queries locally against known/test data sources when prototyping a Beam pipeline with SQL, before productionizing it on remote cluster/services.

First, let's load the `beam_sql` magic:

In [ ]:
%load_ext apache_beam.runners.interactive.sql.beam_sql_magics

Since SQL support in Beam Python SDK is implemented through xLang external transform, make sure you have below prerequisites:
- Have `docker` installed;
- Have jdk8 or jdk11 installed and $JAVA_HOME set;

In [ ]:
!docker image list
!java --version
!echo $JAVA_HOME

In [ ]:
# Optionally sets the logging level to reduce distraction.
import logging

logging.root.setLevel(logging.ERROR)

**Important**: if you're using Beam built from your local source code, additionally:

- Have the Java expansion service shadowjar built. Go to the root directory of your local beam repo and then execute:
  `./gradlew :sdks:java:extensions:sql:expansion-service:shadowJar`.
- Based on your jdk version, pull the docker image `docker pull apache/beam_java11_sdk` or `docker pull apache/beam_java8_sdk`.
- Then tag the image with your current Beam dev version.  You can check the dev version under `apache_beam.version.__version__`. For example, if you're using jdk11 and dev version is `x.x.x.dev`, execute `docker image tag apache/beam_java11_sdk:latest apache/beam_java11_sdk:x.x.x.dev`.

## Query#1 - A simple static query

The `beam_sql` magic can be used as either a line magic or a cell magic.
You can check its usage by running:

In [ ]:
%beam_sql -h

You can run a simple SQL query (in Apache Calcite SQL [syntax](https://beam.apache.org/documentation/dsls/sql/calcite/query-syntax/)) to create a [schema-aware PCollection](https://beam.apache.org/documentation/programming-guide/#schemas) from static values.

In [ ]:
%%beam_sql -o query1_data
SELECT CAST(5 AS INT) AS `id`, CAST('foo' AS VARCHAR) AS `str`, CAST(3.14 AS DOUBLE) AS `flt`

The `beam_sql` magic shows you the result of the SQL query.

It also creates and outputs a PCollection named `query1_data` with `element_type` like `BeamSchema_...(id: int32, str: str)`.

Note that you have **not** explicitly created a Beam pipeline. You get a PCollection because the `beam_sql` magic always **implicitly creates** a pipeline to execute your SQL query. To hold the elements with each field's type info, Beam automatically creates a schema as the `element_type` for the created PCollection.

To introspect the data again with more knobs, you can use `show`.

In [ ]:
from apache_beam.runners.interactive import interactive_beam as ib
ib.show(query1_data)
# Uncomment below to set more args.
# ib.show(query1_data, visualize_data=True, include_window_info=True)

To materialize the PCollection into a pandas [DataFrame](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html#dataframe) object, you can use `collect`.

In [ ]:
ib.collect(query1_data)

You can also additionally append some transforms such as writing to a text file and print the elements:

In [ ]:
import apache_beam as beam

coder=beam.coders.registry.get_coder(query1_data.element_type)
print(coder)
query1_data | beam.io.textio.WriteToText('/tmp/query1_data', coder=coder)
query1_data | beam.Map(print)

Execute the pipeline as a normal pipeline running on DirectRunner and inspect the output file.

In [ ]:
!rm -rf /tmp/query1_data*
query1_data.pipeline.run().wait_until_finish()
!ls /tmp/query1_data*
!cat /tmp/query1_data*

The coder in use is a `RowCoder`. The element is encoded and written to the text file. When inspecting it directly, it may display garbled strings. The file will be revisited later in Query#4.

### [Optional] Omit the `-o` option.
If the option is omitted, an output name is auto-generated based on the SQL query and PCollection (if any) it queries. Optionally, you can also use the `_[{execution_count}]` convention: `_` for last output and `_{execution_count}` for a specific cell execution output.

However, explicitly naming the output is recommended for better notebook readability and to avoid unexpected errors.

Below example outputs a PCollection named like `sql_output_...`.

In [ ]:
%%beam_sql
SELECT CAST(1 AS INT) AS `id`, CAST('foo' AS VARCHAR) AS `str`, CAST(3.14 AS DOUBLE) AS `flt`

Now that you are familiar with the `beam_sql` magic, you can build more queries against PCollections.

Let's install the `names` package to randomly generate some names.

In [ ]:
%pip install names

Import all modules needed for this example.

In [ ]:
import names
import typing

import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
from apache_beam.runners.interactive import interactive_beam as ib

Create a pipeline `p` with the `InteractiveRunner`.

In [ ]:
p = beam.Pipeline(InteractiveRunner())

Then let's create a schema with `typing.NamedTuple`. Let's call it `Person` with a field `id` and a field `name`.

In [ ]:
class Person(typing.NamedTuple):
    id: int
    name: str

With `beam_sql` magic, you can utilize all the Beam I/O connectors (streaming is currently not supported due to `DirectRunner` not supporting streaming pipeline with `SqlTransform`) as source of data, then build a SQL query against all the data and check the output. If needed, you can sink the output following the `WriteToText` example demonstrated above.

## Query#2 - Querying a single PCollection

Let's build a PCollection with 10 random `Person` typed elements.

In [ ]:
persons = (p 
           | beam.Create([Person(id=x, name=names.get_full_name()) for x in range(10)]))
ib.show(persons)

You can look for all elements with `id < 5` in `persons` with the below query and assign the output to `persons_id_lt_5`. Also, you can enable `-v` option to see more details about the execution.

In [ ]:
%%beam_sql -o persons_id_lt_5 -v
SELECT * FROM persons WHERE id <5

With `-v`, if it's the first time running this query, you might see a warning message about

```
Schema Person has not been registered to use a RowCoder. Automatically registering it by running: beam.coders.registry.register_coder(Person, beam.coders.RowCoder)
```

The `beam_sql` magic helps registering a `RowCoder` for each schema you define and use whenever it finds one. You can also explicitly run the same code to do so.

Note the output element type is `Person(id: int, name: str)` instead of `BeamSchema_...` because you have selected all the fields from a single PCollection of the known type `Person(id: int, name: str)`.

## Query#3 - Joining multiple PCollections

You can build a `persons_2` PCollection with a different range of `id`s and `name`s. 

In [ ]:
persons_2 = (p 
             | beam.Create([Person(id=x, name=names.get_full_name()) for x in range(5, 15)]))
ib.show(persons_2)

Then query for all `name`s from `persons` and `persons_2` with the same `id`s and assign the output to `persons_with_common_id`.

In [ ]:
%%beam_sql -o persons_with_common_id -v
SELECT * FROM persons JOIN persons_2 USING (id)

Note the output element type is now some `BeamSchema_...(id: int64, name: str, name0: str)`. Because you have selected columns from both PCollections, there is no known schema to hold the result. Beam automatically creates a schema and differentiates conflicted field `name` by suffixing `0` to one of them.

And since `Person` is already previously registered with a `RowCoder`, there is no more warning about registering it anymore even with `-v`.

## Query#4 - Join multiple PCollections, including I/O.

Let's read the file written by Query#1 and use it to join `persons` and `persons_2` to find `name`s with the common `id` in all three of them. 

In [ ]:
# Use the exact same coder used when WriteToText and explicitly set the output types.
query1_result_in_file = p | beam.io.ReadFromText(
    '/tmp/query1_data*', coder=coder).with_output_types(
    query1_data.element_type)

# Check all the data sources.
ib.show(query1_result_in_file)
ib.show(persons)
ib.show(persons_2)

In [ ]:
%%beam_sql -o entry_with_common_id

SELECT query1_result_in_file.id, persons.name AS `name_1`, persons_2.name AS `name_2`
FROM query1_result_in_file JOIN persons ON query1_result_in_file.id = persons.id
JOIN persons_2 ON query1_result_in_file.id = persons_2.id

You can also chain another `beam_sql` magic to get just `name_1`:

In [ ]:
%%beam_sql -o name_found
SELECT name_1 AS `name` FROM entry_with_common_id